In [1]:
import pandas as pd
import numpy
from __future__ import division

In [2]:
df3_WS4T = pd.read_csv("/Users/asteriag/Desktop/DATASCIENCEPROJECT_WORKFLOW/TimeStamp_Engineering/engineering_AF_segment3_workflowS4_has_duration.csv",engine='python')

In [3]:
df3_WS4T.head(n=1)

,CASE_ID,WRKFLW_TSK_MAP_GROUPED,WRKFLW_TSK_MAP_DETAILED,PATH_CRDATETIME,time_used_str,time_used_second
0,1076747|438001|677047,Release Offer,Release Offer,15/1/2019 17:25:38,1 days 1 hours 13 minutes 3 seconds,90783.0


In [4]:
df3_WS4T['WRKFLW_TSK_MAP_GROUPED'].nunique()

27

In [5]:
df3_WS4T['WRKFLW_TSK_MAP_DETAILED'].nunique()

30

In [6]:
wd_count = df3_WS4T.groupby('WRKFLW_TSK_MAP_DETAILED').WRKFLW_TSK_MAP_DETAILED.count()
wd_dict = dict()
for i, v in wd_count.items():
    wd_dict[i] = v
sorted_wd_dict = sorted(wd_dict.items(), key = lambda x: x[1], reverse = True)
print(sorted_wd_dict)

[('Release Offer', 241), ('Accept', 127), ('Defer', 21), ('Decline', 20), ('Manage Acceptance', 12), ('OC - Satisfied - English language requirements', 12), ('Assessment Decision - Qualified', 9), ('OC - Created - Evidence of course completion', 9), ('OC - Satisfied - Evidence of course completion', 8), ('AR - Satisfied - Academic results for prior tertiary studies', 5), ('AR - Created - Assessment information (Coursework) (1)', 4), ('AR - Satisfied - Official syllabus or subject descriptions (in English)', 4), ('Assessment Decision - No Decision', 4), ('AR - Satisfied - English language proficiency', 3), ('OC - Reviewed - English language requirements', 3), ('AR - Satisfied - Graduate Access Melbourne', 2), ('OC - Created - English language requirements', 2), ('PREPARE TO LOCK', 2), ('AR - Reviewed - English language proficiency', 1), ('AR - Satisfied - Advanced Standing', 1), ('AR - Satisfied - Current enrolment at the University of Melbourne', 1), ('AR - Satisfied - Further informat

In [7]:
list = df3_WS4T.WRKFLW_TSK_MAP_DETAILED.unique()
avg_time_dict = dict()
for key in list :
    df_slides = df3_WS4T[df3_WS4T['WRKFLW_TSK_MAP_DETAILED'] == key]
    sum = df_slides['time_used_second'].sum()
    c = len(df_slides)
    avg = sum//c
    avg_time_dict[key] = avg

sorted_avg_time_dict = sorted(avg_time_dict.items(),key = lambda x: x[1], reverse = True)
print(sorted_avg_time_dict)

[('Put Assessment On-Hold', 18748931.0), ('OC - Created - English language requirements', 5118911.0), ('AR - Created - Assessment information (Coursework) (1)', 4560377.0), ('OC - Satisfied - Evidence of course completion', 4026632.0), ('AR - Satisfied - Academic results for prior tertiary studies', 4014655.0), ('OC - Created - Evidence of course completion', 2989065.0), ('OC - Satisfied - MYEQUALS transcript authentication for students', 1899310.0), ('Release Offer', 1635400.0), ('AR - Satisfied - Further information (1)', 1552547.0), ('Assessment Decision - No Decision', 1276652.0), ('AR - Satisfied - English language proficiency', 980685.0), ('OC - Satisfied - English language requirements', 930477.0), ('Assessment Decision - Qualified', 883029.0), ('AR - Satisfied - Official syllabus or subject descriptions (in English)', 793990.0), ('PREPARE TO LOCK', 319306.0), ('AR - Satisfied - Graduate Access Melbourne', 45616.0), ('Back to Perform Assessment', 36469.0), ('Decline', 17003.0), 

In [8]:
numOfWorkflowProcesses = df3_WS4T.groupby('CASE_ID').CASE_ID.count()

count_dic=dict()
for i,v in numOfWorkflowProcesses.items():
    if count_dic.get(v) is None:
        count_dic[v]=[]
        count_dic[v].append(i)
    else:
        count_dic[v].append(i)
#count_dic
count_dic = dict(sorted(count_dic.items()))

In [9]:
len_list=[]
for key in count_dic.keys():
    len_list.append(len(count_dic[key]))

In [10]:
def strfdelta(tdelta, fmt):
    d=dict()
    d["days"],rem1 = divmod(tdelta,60*60*24)
    d["hours"], rem2 = divmod(rem1, 3600)
    d["minutes"], d["seconds"] = divmod(rem2, 60)
    return fmt.format(**d)

In [11]:
list_avgtime_per_key =[]
list_sumtime_row = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_person = 0
    for id in id_list:
        df = df3_WS4T[df3_WS4T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_person+=tsum
    list_avgtime_per_key.append(int((list_sumtime_person/len(count_dic[key]))))
    list_sumtime_row.append(list_sumtime_person)
list_avgtime_per_key_str = []
for i in list_avgtime_per_key:
    list_avgtime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [12]:
list_mediantime_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_per_person = []
    for id in id_list:
        df = df3_WS4T[df3_WS4T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_per_person.append(tsum)
    list_mediantime_per_key.append(numpy.median(list_sumtime_per_person))

list_mediantime_per_key_str = []
for i in list_mediantime_per_key:
    list_mediantime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [13]:
list_percent_reach_end = []
for key in count_dic.keys():
    id_list = count_dic[key]
    count_df_end_step = 0
    count_df_id = 0
    for id in id_list:
        df_id = df3_WS4T[df3_WS4T['CASE_ID']== id]
        df_end_step = df_id[df_id['WRKFLW_TSK_MAP_GROUPED'].isin(['Accept','Defer'])]
        count_df_id += df_id['CASE_ID'].nunique()
        count_df_end_step += df_end_step['CASE_ID'].nunique()
    list_percent_reach_end.append(str(round((count_df_end_step/count_df_id)*100,2))+ "%")


In [14]:
list_step_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = set()
    for id in id_list:
        df = df3_WS4T[df3_WS4T['CASE_ID']== id]
        for index, row in df.iterrows():
            list_step.add(row['WRKFLW_TSK_MAP_GROUPED'])
    list_step_per_key.append(list_step)

In [15]:
from collections import Counter
import collections, functools, operator
list_step_sum_time_per_key = []
list_step_count_per_key = []
list_step_ave_time_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = []
    step_time_list = []
    for id in id_list:
        df = df3_WS4T[df3_WS4T['CASE_ID']== id]
        step_time = dict(zip(df.WRKFLW_TSK_MAP_DETAILED, df.time_used_second))
        step_time_list.append(step_time)
        for index, row in df.iterrows():
            list_step.append(row['WRKFLW_TSK_MAP_DETAILED'])

    step_sum_time = dict(functools.reduce(operator.add,
         map(collections.Counter, step_time_list)))

    dict_list_step=dict(Counter(list_step))

    step_ave_time = {key: step_sum_time[key] // dict_list_step.get(key, 0)
                        for key in step_sum_time.keys()}

    sort_step_sum_time = sorted(step_sum_time.items(),key = lambda x: x[1], reverse = True)
    list_step_sum_time_per_key.append(sort_step_sum_time)

    sort_dict_list_step = sorted(dict_list_step.items(),key = lambda x: x[1], reverse = True)
    list_step_count_per_key.append(sort_dict_list_step[:6])

    sort_step_ave_time = sorted(step_ave_time.items(),key = lambda x: x[1], reverse = True)
    list_step_ave_time_per_key.append(sort_step_ave_time)

In [16]:
s3_t4=pd.DataFrame({'Number of Workflow processes in this Workflow Segment':count_dic.keys(),'Number of cases':len_list, 'average time for workflow segment':list_avgtime_per_key_str,'median time for workflow segment':list_mediantime_per_key_str,'percentage of cases reaching to end of workflow segment':list_percent_reach_end})
s3_t4['Num of Occurrences per step(descending)'] = list_step_count_per_key
s3_t4['Average time spends per step(descending)'] = list_step_ave_time_per_key
s3_t4['average time for workflow segment(seconds)'] = list_avgtime_per_key
s3_t4['sum time for workflow segment(seconds)'] = list_sumtime_row
s3_t4.head(n=10)


,Number of Workflow processes in this Workflow Segment,Number of cases,average time for workflow segment,median time for workflow segment,percentage of cases reaching to end of workflow segment,Num of Occurrences per step(descending),Average time spends per step(descending),average time for workflow segment(seconds),sum time for workflow segment(seconds)
0,1,56,0 days 0 hours 0 minutes 0 seconds,0.0 days 0.0 hours 0.0 minutes 0.0 seconds,0.0%,"[(Release Offer, 56)]",[],0,0.0
1,2,137,27 days 15 hours 43 minutes 24 seconds,9.0 days 6.0 hours 28.0 minutes 1.0 seconds,81.02%,"[(Release Offer, 137), (Accept, 98), (Decline,...","[(Release Offer, 2389404.0)]",2389404,327348427.0
2,3,30,50 days 22 hours 19 minutes 44 seconds,26.0 days 2.0 hours 59.0 minutes 7.5 seconds,83.33%,"[(Release Offer, 30), (Accept, 21), (Manage Ac...",[(OC - Satisfied - Evidence of course completi...,4400384,132011541.0
3,4,10,55 days 21 hours 18 minutes 17 seconds,28.0 days 13.0 hours 28.0 minutes 29.0 seconds,70.0%,"[(Release Offer, 10), (Accept, 4), (OC - Creat...","[(Put Assessment On-Hold, 18748931.0), (AR - C...",4828697,48286973.0
4,5,3,76 days 17 hours 26 minutes 35 seconds,43.0 days 6.0 hours 55.0 minutes 6.0 seconds,100.0%,"[(Release Offer, 3), (Assessment Decision - Qu...",[(AR - Satisfied - Academic results for prior ...,6629195,19887586.0
5,6,3,94 days 7 hours 21 minutes 56 seconds,59.0 days 2.0 hours 41.0 minutes 19.0 seconds,33.33%,"[(Release Offer, 3), (Assessment Decision - No...",[(OC - Created - Evidence of course completion...,8148116,24444349.0
6,7,1,40 days 2 hours 45 minutes 12 seconds,40.0 days 2.0 hours 45.0 minutes 12.0 seconds,100.0%,"[(Release Offer, 2), (OC - Satisfied - English...",[(AR - Satisfied - Academic results for prior ...,3465912,3465912.0


In [17]:
s3_t4.to_csv('engineering_AF_segment3_workflowS4_table.csv', index=False)


